## Perform Daily Inventory and Price Data Download

In [2]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import glob
import os.path
import shutil
import keyring
from datetime import datetime

In [8]:


username = "jmeisenh"
password = keyring.get_password('magic', 'jmeisenh')
options = Options()
options.binary_location = "C:/Program Files/Mozilla Firefox/firefox.exe"

PATH = "C:/Users/jmeis/Projects/MagicPrice/geckodriver.exe"
driver = webdriver.Firefox(executable_path=PATH, options = options)

driver.get("https://deckbox.org/accounts/login?return_to=/sets/3285758")
driver.maximize_window()
# Find username, wait and send username
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "login"))).send_keys(username)

# find password input field and insert password
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(password)

# click login button
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "submit_button"))).click()
#driver.find_element_by_id("submit_button").click()
time.sleep(2)
a = ActionChains(driver)
#activate tools hover menu
m = driver.find_element_by_link_text("Tools")
a.move_to_element(m).perform()
#click export
n = driver.find_element_by_link_text("Export")
a.move_to_element(n).click().perform()
time.sleep(1)

# click on extra columns button to expand list
driver.find_element_by_id("extra_columns_container").click()
time.sleep(1)
driver.switch_to_default_content()
# click on price
driver.find_element_by_link_text("Price").click()
#click on rarity
driver.find_element_by_link_text("Rarity").click()
# click on type
driver.find_element_by_xpath("/html/body/div[12]/div[1]/ul/li[1]/a").click()
# click on cost
driver.find_element_by_xpath("/html/body/div[12]/div[1]/ul/li[2]/a").click()
# done button
driver.find_element_by_xpath("/html/body/div[12]/div[2]/div").click()
time.sleep(1)
# finish export
driver.find_element_by_css_selector("button.btn-primary:nth-child(7)").click()
# quit instance
time.sleep(3)
driver.quit()

# identify new exported file
src_path = r'C:\Users\jmeis\Downloads'
file_type = r'\*csv'
files = glob.glob(src_path + file_type)
max_file = max(files, key=os.path.getctime)

#move file to working directory
file_name = os.path.basename(max_file)
dst_path = r'C:\Users\jmeis\Projects\MagicPrice\Exports'
shutil.move(max_file, dst_path+ '\\' +datetime.today().strftime('%Y-%m-%d') + '.csv')










C:\Users\jmeis\AppData\Local\Temp\ipykernel_9540\481495847.py:47: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()


'C:\\Users\\jmeis\\Projects\\MagicPrice\\Exports\\2023-02-27.csv'

## Convert File to Useable Format

In [69]:
# Import Data
importDir = r'C:\Users\jmeis\Projects\MagicPrice\Exports' + '\\' + datetime.today().strftime('%Y-%m-%d') + '.csv'
df = pd.read_csv(importDir)
# Keep only usable columns
df = df[['Count', 'Name', 'Edition', 'Card Number', 'Condition', 'Language', 'Foil', 'Type', 'Cost', 'Rarity', 'Price']]

# Add Color Column
#eliminate duplicates in Color Column
df['Color'] = df['Cost'].astype('string')
# fill nas with temporary value
df['Color'] = df.Color.fillna('Z')
df['Color'] = df.Color.apply(lambda x: "".join(sorted(set(x))))

# Replace brackets, X , C, and phyrexian mana with space
for i in ['[^A-Z]', 'X', 'P', 'C']:
    df['Color'] = df['Color'].str.replace(i, '')

#create color replacement dictionary

colorDict = {
    r'\bB\b' : 'Black',
    r'\bU\b' : 'Blue', 
    r'\bG\b' : 'Green', 
    r'\bR\b' : 'Red',
    r'\bW\b' : 'White',
    r'\b[BU]{2}\b' : 'Dimir',
    r'\b[BG]{2}\b' : 'Golgari',
    r'\b[BR]{2}\b' : 'Rakdos',
    r'\b[BW]{2}\b' : 'Orzhov',
    r'\b[UG]{2}\b' : 'Simic',
    r'\b[UR]{2}\b' : 'Izzit',
    r'\b[UW]{2}\b' : 'Azorius',
    r'\b[GR]{2}\b' : 'Gruul',
    r'\b[GW]{2}\b' : 'Selesnya',
    r'\b[RW]{2}\b' : 'Boros',
    r'\b[UGW]{3}\b' : 'Bant',
    r'\b[WBR]{3}\b' : 'Mardu',
    r'\b[UBR]{3}\b' : 'Grixis',
    r'\b[BRG]{3}\b' : 'Jund',
    r'\b[RGW]{3}\b' : 'Naya',
    r'\b[WBG]{3}\b' : 'Abzan',
    r'\b[URW]{3}\b' : 'Jeskai',
    r'\b[BGU]{3}\b' : 'Sultai',
    r'\b[GUR]{3}\b' : 'Temur',
    r'\b[WUB]{3}\b' : 'Esper',
    r'\b[WUBR]{4}\b' : 'Yore-Tiller',
    r'\b[UBRG]{4}\b' : 'Glint-Eye',
    r'\b[BRGW]{4}\b' : 'Dune-Brood',
    r'\b[RGWU]{4}\b' : 'Ink-Treader',
    r'\b[GWUB]{4}\b' : 'Witch-Maw',
    r'\b[WUBRG]{5}\b' : 'Dominion',
    r'\bZ\b' : 'Colorless'
    
}
for key, value in colorDict.items():
    df['Color'] = df['Color'].str.replace(key, value)
# all remaining empty cells will be colorless
df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')
# Save file
dst_path = r'C:\Users\jmeis\Projects\MagicPrice\Exports'
df.to_csv(dst_path + '\\' + 'Cleaned_'+ datetime.today().strftime('%Y-%m-%d') + 'csv', index = False)




C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\617386118.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(i, '')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\617386118.py:56: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(key, value)
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\617386118.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')


## Convert all old exports to useable format

In [75]:
directory = r'C:\Users\jmeis\Projects\MagicPrice\Exports\temp'
for i in os.listdir(directory):
        # Import Data
    print(directory + '\\' + i)
    df = pd.read_csv(directory + '\\' + i)
    # Keep only usable columns
    df = df[['Count', 'Name', 'Edition', 'Card Number', 'Condition', 'Language', 'Foil', 'Type', 'Cost', 'Rarity', 'Price']]

    # Add Color Column
    #eliminate duplicates in Color Column
    df['Color'] = df['Cost'].astype('string')
    # fill nas with temporary value
    df['Color'] = df.Color.fillna('Z')
    df['Color'] = df.Color.apply(lambda x: "".join(sorted(set(x))))

    # Replace brackets, X , C, and phyrexian mana with space
    for j in ['[^A-Z]', 'X', 'P', 'C']:
        df['Color'] = df['Color'].str.replace(j, '')

    #create color replacement dictionary

    colorDict = {
        r'\bB\b' : 'Black',
        r'\bU\b' : 'Blue', 
        r'\bG\b' : 'Green', 
        r'\bR\b' : 'Red',
        r'\bW\b' : 'White',
        r'\b[BU]{2}\b' : 'Dimir',
        r'\b[BG]{2}\b' : 'Golgari',
        r'\b[BR]{2}\b' : 'Rakdos',
        r'\b[BW]{2}\b' : 'Orzhov',
        r'\b[UG]{2}\b' : 'Simic',
        r'\b[UR]{2}\b' : 'Izzit',
        r'\b[UW]{2}\b' : 'Azorius',
        r'\b[GR]{2}\b' : 'Gruul',
        r'\b[GW]{2}\b' : 'Selesnya',
        r'\b[RW]{2}\b' : 'Boros',
        r'\b[UGW]{3}\b' : 'Bant',
        r'\b[WBR]{3}\b' : 'Mardu',
        r'\b[UBR]{3}\b' : 'Grixis',
        r'\b[BRG]{3}\b' : 'Jund',
        r'\b[RGW]{3}\b' : 'Naya',
        r'\b[WBG]{3}\b' : 'Abzan',
        r'\b[URW]{3}\b' : 'Jeskai',
        r'\b[BGU]{3}\b' : 'Sultai',
        r'\b[GUR]{3}\b' : 'Temur',
        r'\b[WUB]{3}\b' : 'Esper',
        r'\b[WUBR]{4}\b' : 'Yore-Tiller',
        r'\b[UBRG]{4}\b' : 'Glint-Eye',
        r'\b[BRGW]{4}\b' : 'Dune-Brood',
        r'\b[RGWU]{4}\b' : 'Ink-Treader',
        r'\b[GWUB]{4}\b' : 'Witch-Maw',
        r'\b[WUBRG]{5}\b' : 'Dominion',
        r'\bZ\b' : 'Colorless'

    }
    for key, value in colorDict.items():
        df['Color'] = df['Color'].str.replace(key, value)
    # all remaining empty cells will be colorless
    df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')
    # Save file
    dst_path = r'C:\Users\jmeis\Projects\MagicPrice\Exports'
    df.to_csv(dst_path + '\\' + 'Cleaned_' + i  , index = False)




C:\Users\jmeis\Projects\MagicPrice\Exports\temp\2023-02-01.csv


C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(j, '')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(key, value)
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(j, '')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:58: FutureWarning: The default value of regex will change from True to False in a fu

C:\Users\jmeis\Projects\MagicPrice\Exports\temp\2023-02-03.csv


C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(j, '')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(key, value)


C:\Users\jmeis\Projects\MagicPrice\Exports\temp\2023-02-08.csv


...

C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(j, '')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(key, value)


C:\Users\jmeis\Projects\MagicPrice\Exports\temp\2023-03-07.csv


C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(j, '')
C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(key, value)


C:\Users\jmeis\Projects\MagicPrice\Exports\temp\2023-03-08.csv


C:\Users\jmeis\AppData\Local\Temp\ipykernel_13812\957531585.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Color'] = df['Color'].str.replace(r'^\s*$', 'Colorless')


## Create Current long format card and price listing

In [166]:
dst_path = r'C:\Users\jmeis\Projects\MagicPrice\Exports'
df = pd.read_csv(dst_path + '\\' + 'Cleaned_'+ datetime.today().strftime('%Y-%m-%d') + '.csv')
directory = r'C:\Users\jmeis\Projects\MagicPrice\Exports\Temp'
for i in os.listdir(directory):
    oldDF = pd.read_csv(directory + '\\' + i)
    mergeDF = pd.merge(df, oldDF, how = 'left', on = ['Name', 'Edition', 'Card Number', 'Condition', 'Language', 'Foil'], suffixes=('', '_remove'))
    mergeDF.rename(columns = {'Count_remove' : i.replace('Cleaned_', '').replace('.csv', '')}, inplace = True)
    mergeDF.rename(columns = {'Price_remove' : i.replace('Cleaned_', '').replace('.csv', '')}, inplace = True)
    mergeDF.drop([j for j in mergeDF.columns if 'remove' in j],axis=1, inplace=True)
    df = mergeDF
df = df[['Count', 'Name', 'Edition', 'Card Number', 'Condition', 'Language',
       'Foil', 'Type', 'Cost', 'Rarity', 'Color', '2022-12-22',
       '2022-12-23', '2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27',
       '2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31', '2023-01-01',
       '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-19', '2023-01-20',
       '2023-01-21', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26',
       '2023-01-27', '2023-01-28', '2023-01-29', '2023-01-30', '2023-02-01',
       '2023-02-03', '2023-02-08', '2023-02-10', '2023-02-11', '2023-02-12',
       '2023-02-13', '2023-02-14', '2023-02-16', '2023-02-17', '2023-02-18',
       '2023-02-19', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-27',
       '2023-02-28', '2023-03-02', '2023-03-03', '2023-03-04', '2023-03-05',
       '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10','Price']]
df.rename(columns = {'Price': datetime.today().strftime('%Y-%m-%d')}, inplace = True)

df.to_csv(dst_path + '\\' + 'AllCardsWide.csv', index = False)
pd.melt(df, id_vars=list(df.columns)[0:11] , value_vars=list(df.columns)[11:], 
                     var_name= 'Date', value_name='Price', ignore_index=False).sort_values(['Name', 'Date']).to_csv(dst_path + '\\' + 'AllCardsLong.csv', index = False)    
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jmeis\\Projects\\MagicPrice\\Exports\\Cleaned_2023-03-11.csv'

## Convert previous code to scheduled task script

In [179]:
AllCardsWide =  pd.read_csv(dst_path + '\\' + 'AllCardsWide.csv')
# The following line will not be necesarry in script (df is already defined)
df = pd.read_csv(dst_path + '\\' + 'Cleaned_' + datetime.today().strftime('%Y-%m-%d') + '.csv')
# merge dfs
mergeDF = pd.merge(AllCardsWide, df, how = 'left', on = ['Name', 'Edition', 'Card Number', 'Condition', 'Language', 'Foil'], suffixes=('', '_remove'))
# drop original count column, and rename new count column
mergeDF.drop('Count', axis = 1, inplace = True)
# rename count_remove to count and move to first column
mergeDF.rename(columns = {'Count_remove' : 'Count'}, inplace = True)
mergeDF = mergeDF[['Count'] + [x for x in mergeDF.columns if x != 'Count']]
# rename newest price column and move to end
mergeDF.rename(columns = {'Price' : datetime.today().strftime('%Y-%m-%d')}, inplace = True)
cols = list(mergeDF.columns.values) 
cols.pop(cols.index(datetime.today().strftime('%Y-%m-%d'))) 
mergeDF = mergeDF[cols+[datetime.today().strftime('%Y-%m-%d')]] 
# remove all right hand columns

mergeDF.drop([j for j in mergeDF.columns if 'remove' in j],axis=1, inplace=True)
#mergeDF.drop([j for j in mergeDF.columns if '.1' in j],axis=1, inplace=True)
# save new wideformat csv
mergeDF.to_csv(dst_path + '\\' + 'AllCardsWide.csv', index = False)
# Save new long format csv
pd.melt(mergeDF, id_vars=list(mergeDF.columns)[0:11] , value_vars=list(mergeDF.columns)[11:], 
                     var_name= 'Date', value_name='Price', ignore_index=False).sort_values(['Name', 'Date']).to_csv(dst_path + '\\' + 'AllCardsLong.csv', index = False) 



In [180]:
list(mergeDF.columns)

['Count',
 'Name',
 'Edition',
 'Card Number',
 'Condition',
 'Language',
 'Foil',
 'Type',
 'Cost',
 'Rarity',
 'Color',
 '2022-12-22',
 '2022-12-23',
 '2022-12-24',
 '2022-12-25',
 '2022-12-26',
 '2022-12-27',
 '2022-12-28',
 '2022-12-29',
 '2022-12-30',
 '2022-12-31',
 '2023-01-01',
 '2023-01-02',
 '2023-01-03',
 '2023-01-04',
 '2023-01-19',
 '2023-01-20',
 '2023-01-21',
 '2023-01-23',
 '2023-01-24',
 '2023-01-25',
 '2023-01-26',
 '2023-01-27',
 '2023-01-28',
 '2023-01-29',
 '2023-01-30',
 '2023-02-01',
 '2023-02-03',
 '2023-02-08',
 '2023-02-10',
 '2023-02-11',
 '2023-02-12',
 '2023-02-13',
 '2023-02-14',
 '2023-02-16',
 '2023-02-17',
 '2023-02-18',
 '2023-02-19',
 '2023-02-21',
 '2023-02-22',
 '2023-02-23',
 '2023-02-27',
 '2023-02-28',
 '2023-03-02',
 '2023-03-03',
 '2023-03-04',
 '2023-03-05',
 '2023-03-07',
 '2023-03-08',
 '2023-03-09',
 '2023-03-10',
 '2023-03-11']

In [173]:
mergeDF

,Count,Name,Edition,Card Number,Condition,Language,Foil,Type,Cost,Rarity,...,2023-03-08,2023-03-09,Type_remove,Cost_remove,Rarity_remove,2023-03-11,2023-03-11,Color_remove,2023-03-11,2023-03-11
0,2,A Little Chat,Streets of New Capenna,47,Near Mint,English,NaN,Instant,{1}{U},Uncommon,...,$0.07,$0.07,Instant,{1}{U},Uncommon,$0.07,$0.07,Blue,$0.07,$0.07
1,3,Abandon Reason,Eldritch Moon,115,Near Mint,English,NaN,Instant,{2}{R},Uncommon,...,$0.14,$0.14,Instant,{2}{R},Uncommon,$0.14,$0.14,Red,$0.14,$0.14
2,2,Abandoned Sarcophagus,Commander 2020,236,Near Mint,English,NaN,Artifact,{3},Rare,...,$0.14,$0.14,Artifact,{3},Rare,$0.14,$0.14,Colorless,$0.14,$0.14
3,1,Abattoir Ghoul,Innistrad,85,Near Mint,English,NaN,Creature - Zombie,{3}{B},Uncommon,...,$0.09,$0.09,Creature - Zombie,{3}{B},Uncommon,$0.09,$0.09,Black,$0.09,$0.09
4,1,"Abdel Adrian, Gorion's Ward",Commander Legends: Battle for Baldur's Gate,375,Near Mint,English,foil,Legendary Creature - Human Warrior,{4}{W},Uncommon,...,$0.22,$0.22,Legendary Creature - Human Warrior,{4}{W},Uncommon,$0.22,$0.23,White,$0.22,$0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16173,1,Zulaport Cutthroat,Commander Legends: Battle for Baldur's Gate,775,Near Mint,English,NaN,Creature - Human Rogue Ally,{1}{B},Uncommon,...,$0.62,$0.57,Creature - Human Rogue Ally,{1}{B},Uncommon,$0.56,$0.59,Black,$0.56,$0.59
16174,1,Zulaport Cutthroat,Battle for Zendikar,126,Near Mint,English,foil,Creature - Human Rogue Ally,{1}{B},Uncommon,...,$4.62,$4.62,Creature - Human Rogue Ally,{1}{B},Uncommon,$4.67,$4.76,Black,$4.67,$4.76
16175,1,Zur the Enchanter,Coldsnap,135,Near Mint,English,NaN,Legendary Creature - Human Wizard,{1}{W}{U}{B},Rare,...,$3.19,$3.19,Legendary Creature - Human Wizard,{1}{W}{U}{B},Rare,$3.19,$3.13,Esper,$3.19,$3.13
16176,1,Zuran Orb,Ice Age,325,Near Mint,English,NaN,Artifact,{0},Uncommon,...,$2.57,$2.62,Artifact,{0},Uncommon,$2.62,$2.62,Colorless,$2.62,$2.62
